# Stock Price Dynamics

Modeling stock price dynamics is an important aspect of mathematical finance. Simulating stock price movements allows an algorithmic trader to test different strategies. Policy makers at the federal reserve can evaluate risk and volatility using similar models.

It's important to keep in mind that every financial model contains assumptions and limitations. Understanding these limitations is a crucial part of being a responsible practicioner. At any rate, stochastic models are still a valuable tool for assessing risk and volatility.

## Geometric Brownian Motion

Simulations of stocks and options are often modeled using stochastic differential equations (SDEs). Because of the randomness associated with stock price movements, the models cannot be developed using ordinary differential equations (ODEs).

A typical model used for stock price dynamics is the following stochastic differential equation:

$$dS = \mu S dt + \sigma S dW$$